In [1]:
import sys
import os
import pandas as pd
# 현재 작업 디렉토리 기준으로 src 디렉토리 추가
notebook_dir = os.getcwd()  # 현재 Jupyter Notebook의 작업 디렉토리

sys.path.append(os.path.dirname(notebook_dir))


In [2]:
from src.tools.google_drive import ensure_data_files

# 데이터 파일 경로 가져오기
data_paths = ensure_data_files()
data_paths


기존 data가 존재합니다. 파일 경로를 반환합니다.


{'reviewer': '/Users/seongrok.kim/Github/yamyam-lab/data/reviewer.csv',
 'review_keyword': '/Users/seongrok.kim/Github/yamyam-lab/data/review_keyword.csv',
 'category': '/Users/seongrok.kim/Github/yamyam-lab/data/diner_category_raw.csv',
 'review': '/Users/seongrok.kim/Github/yamyam-lab/data/review.csv',
 'dinr_menu': '/Users/seongrok.kim/Github/yamyam-lab/data/dinr_menu.csv',
 'diner': '/Users/seongrok.kim/Github/yamyam-lab/data/diner.csv'}

In [3]:
# load data
diner = pd.read_csv(data_paths["diner"])
diner_category = pd.read_csv(data_paths["category"])

diner = pd.merge(diner, diner_category, on="diner_idx", how="left")


review = pd.read_csv(data_paths["review"])
reviewer = pd.read_csv(data_paths["reviewer"])

review = pd.merge(review, reviewer, on="reviewer_id", how="left")

/var/folders/dq/xrxyf7x57q504wf515d672p00000gn/T/ipykernel_67351/2461593576.py:2: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  diner = pd.read_csv(data_paths["diner"])


In [18]:
import pandas as pd
# 날짜 타입으로 변환
review["reviewer_review_date"] = pd.to_datetime(review["reviewer_review_date"])

# 1. 리뷰어별 리뷰 수 및 작성 기간 계산
agg_df = (
    review.groupby("reviewer_id")["reviewer_review_date"]
    .agg(["count", lambda x: (x.max() - x.min()).days])
    .rename(columns={"count": "review_count", "<lambda_0>": "review_period_days"})
    .reset_index()
)

# 2. 짧은 기간에 리뷰 많이 남긴 리뷰어 추출 (예: 5개 이상 리뷰, 3일 이하 기간)
suspicious_reviewers = agg_df[
    (agg_df["review_count"] >= 3) & (agg_df["review_period_days"] <= 3)
]["reviewer_id"]

# 3. 해당 리뷰어들의 리뷰 중 4점 초과만 필터링
high_score_reviews = review[
    (review["reviewer_id"].isin(suspicious_reviewers))
    & (review["reviewer_review_score"] >= 4) 
    #& (review["reviewer_avg"] >= 4)
]

# 4. 음식점별 리뷰 수 카운트
diner_counts = high_score_reviews["diner_idx"].value_counts()

# 5. 리뷰 수가 10개 이상인 음식점만 추출
target_diners = diner_counts[diner_counts >= 20].index

# 6. 해당 음식점 리뷰만 보기
suspicious_reviews = high_score_reviews[high_score_reviews["diner_idx"].isin(target_diners)]

# 결과 확인
suspicious_reviews = suspicious_reviews.sort_values(["diner_idx", "reviewer_id", "reviewer_review_date"])
suspicious_reviews.head()


,Unnamed: 0,diner_idx,reviewer_id,review_id,reviewer_review,reviewer_review_date,reviewer_review_score,reviewer_user_name,reviewer_review_cnt,reviewer_avg,badge_grade,badge_level
95976,96162,7846414.0,300418083,977494,워낙 맛집 찾아다니면서 웨이팅하는 거 익숙해져서 기다리는 거 괜찮아요 항상 1시간 ...,2019-08-09,5.0,단미보글,3,3.5,브론즈 배지,5
400402,401171,7846414.0,524801172,58442,강릉가면 가보셔도 될 식당이라고 생각합니다! 맛있어요!! 근데 가격이... 9000...,2018-01-26,4.0,솔직헌심정,3,4.5,실버 배지,11
423853,424700,7846414.0,527767989,8315019,이동약자접근 순두부찌개 청국장찌개,2023-08-14,5.0,토토프로토,4,4.9,실버 배지,21
703772,705276,7846414.0,772032333,437915,"짬뽕순두부의 양이 생각외로 많았으며, 순두부가 부족하지 않게 많이 들어 있어 나름 ...",2019-03-09,4.0,쌍마트랜스,4,2.8,브론즈 배지,4
839670,841428,7846414.0,905525705,5020512,NaN,2022-02-08,5.0,유재희,3,4.2,브론즈 배지,5


In [19]:
# 3일 안에 3개 이상을 쓴 리뷰어가 20명 이상 음식점들
suspicious_reviews['diner_idx'].value_counts()

diner_idx
1.154752e+07    50
1.033241e+07    49
8.453998e+06    44
8.025375e+06    43
2.549189e+08    40
1.562029e+09    38
1.578746e+09    38
3.775179e+08    38
5.006726e+08    38
2.145579e+07    35
1.728065e+09    35
4.695770e+08    34
2.752536e+07    33
1.148098e+09    32
1.194111e+09    29
1.037514e+07    26
1.916683e+09    26
1.716327e+07    25
2.504847e+07    25
1.981652e+09    25
2.753103e+07    23
2.544889e+07    22
9.006988e+06    22
1.890778e+09    22
5.872494e+08    21
2.003155e+07    21
1.173308e+07    21
1.280103e+09    20
8.094563e+06    20
7.846414e+06    20
Name: count, dtype: int64

In [24]:
# 1. 음식점별 날짜별 리뷰 수 집계
date_overlap = (
    suspicious_reviews.groupby(["diner_idx", "reviewer_review_date"])
    .size()
    .reset_index(name="review_count_per_date")
)

# 2. 각 음식점에서 가장 많이 겹친 날짜의 리뷰 수 추출
max_overlap_per_diner = (
    date_overlap.groupby("diner_idx")["review_count_per_date"]
    .max()
    .reset_index(name="max_reviews_on_same_date")
)

# 3. 겹치는 수 기준으로 음식점 정렬
sorted_diners = max_overlap_per_diner.sort_values(by="max_reviews_on_same_date", ascending=False)

# 4. suspicious_reviews에 merge해서, 겹친 수 기준 정렬 정보 부여
suspicious_reviews_with_overlap = suspicious_reviews.merge(sorted_diners, on="diner_idx")

# 5. 최종 정렬된 suspicious_reviews 보기 (많이 겹친 순서)
final_result = suspicious_reviews_with_overlap.sort_values(
    by=["max_reviews_on_same_date", "diner_idx", "reviewer_review_date"],
    ascending=[False, True, True]
)

In [25]:
# 한 음식점 내에서 3일 안에 3개 이상을 쓴 리뷰어가 리뷰를 쓴 날짜가 가장 많이 겹친 것들 추출
final_result

,Unnamed: 0,diner_idx,reviewer_id,review_id,reviewer_review,reviewer_review_date,reviewer_review_score,reviewer_user_name,reviewer_review_cnt,reviewer_avg,badge_grade,badge_level,max_reviews_on_same_date
479,2271205,2.549189e+08,3524890375,10388478,가족들과 같이 방문했습니다. 조카도 맛있게 먹었어요,2024-06-20,5.0,김서인,3,4.9,브론즈 배지,7,5
480,2272228,2.549189e+08,3589659113,10422926,"가게도 깔끔하고 시원하고 좋은데, 고기맛까지 기가막혀요... 감동...",2024-06-25,5.0,이선아,4,5.0,브론즈 배지,3,5
481,2286102,2.549189e+08,3753022880,11281890,인생고깃집 오늘 발견했네요 왜 진작에 안왔을까하고 후회했어요 그만큼 너무 맛있었고 ...,2024-10-23,5.0,내가왕이될상인가,3,5.0,브론즈 배지,4,5
482,2286224,2.549189e+08,3754609570,11282057,숯불향이 고기에 잘 배겨서 더 맛있었던거같아요 소양념갈비 부드럽게 맛있습니다ㅎㅎ,2024-10-23,5.0,shut,5,4.9,브론즈 배지,8,5
483,2286655,2.549189e+08,3761234139,11282134,고기먹을땐 항상 여기 명동 고기맛집임,2024-10-23,5.0,안녕자두야,3,4.9,브론즈 배지,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,1329174,1.890778e+09,1209319159,11144355,최고의 돈까스집 \n치즈 너무 고소하고 듬뿍들어있어 좋았고\n등심 촉촉하지만 겉 빵...,2024-10-04,5.0,무무야,3,3.4,실버 배지,16,1
862,2207674,1.890778e+09,2781347164,11464683,드디어 보았다!!\n마음 같아서는 악수라도 하고 싶었다 ㅜㅜ 우엉\n보통 등심은 질...,2024-11-19,5.0,시아,3,4.8,실버 배지,15,1
852,1530555,1.890778e+09,1341611830,11703403,NaN,2024-12-27,5.0,수진,3,4.5,브론즈 배지,5,1
851,1455962,1.890778e+09,1280021324,11939720,솔직히 돈까스 맛집 맞음 맛있어요\n근데 웨이팅이 사악하긴해요\n\n치즈는 드시려면...,2025-01-18,5.0,이상윤,4,4.0,브론즈 배지,10,1
